# Omics Data QA/QC Analysis: 
* **Authors: Nurgul Kaplan, Christopher Petzold, Yan Chen, Jennifer Gin**
* **Authors_EDD Utils:Zak Costello, William Morrell, Mark Forrer, Tijana Radivojevic**
* **Author_Kernel-Jupyter server: Mark Kulawik**
* **Version: 1.07**
* **Date:20201123**



Load necessary python libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from datetime import datetime
import seaborn as sns; sns.set(color_codes=True)
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import HBox, Label, Layout ,Button,AppLayout, jslink, IntText, IntSlider


In [2]:
%matplotlib inline

In [3]:
# Import EDD utils to import study as pandas dataframe
from edd_utils import login, export_study


In [4]:
# Create a function which is for entering the URL of EDD study
def edd_study_url():
     
#     colors:
    class bcolors:
        HEADER = '\033[95m'
        FAIL = '\033[91m'
        BOLD = '\033[1m'
        
    while True:
        try:
            user_edd_study_url = input("Please enter EDD STUDY URL: ").lower()
            user_edd_study_url = user_edd_study_url.replace(" ","")
            if "/s/" not in user_edd_study_url:
                raise ValueError()
            else:
                break
        except ValueError:
            print(f"{bcolors.HEADER}{bcolors.BOLD}PLEASE TRY AGAIN")
            
            
    url_parts = user_edd_study_url.split("/")    
    s_index = url_parts.index("s")
    edd_server = url_parts[s_index-1]
    study_slug = url_parts[s_index+1]
    final_url_parts = [edd_server, study_slug]

    return final_url_parts

***
 **Example of EDD study URL link:**

 https://<font color=blue>edd_server</font>/s/<font color=red>This-is-my-Study-Slug</font>/

***

<font color=blue>**EDD** study url:</font>

 https://<font color=blue>public-edd.agilebiofoundry.org</font>/s/<font color=red>example-data-quality-study</font>/


## Enter <font color=green>**EDD study URL link**</font>, Please see the example shown above


In [5]:
#User input requested/
final_url_parts = edd_study_url()

Please enter EDD STUDY URL:  https://public-edd.agilebiofoundry.org/s/example-data-quality-study/


In [6]:
# Parse EDD STUDY URL for edd_utils
edd_server = final_url_parts[0]
study_slug = final_url_parts[1]

## Enter <font color=green>**LDAP password**</font>

 * session = login(edd_server=edd_server)
 * Password for <font color=red>YourLDAPusername</font>:     <font color=green>**LDAP password**</font>

In [7]:
# Create EDD session
session = login(edd_server=edd_server)


Password for nkaplan:  ················


In [8]:
df = export_study(session, study_slug, edd_server=edd_server)



In [9]:
df.head()

,Study ID,Study Name,Line ID,Line Name,Line Description,Protocol,Assay ID,Assay Name,Formal Type,Measurement Type,Compartment,Units,Value,Hours
0,7342,Example Data Quality Study,7343,Line-1-R1,This is Line-1,Targeted Proteomics,7352,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,0,counts,7209.0,24.0
1,7342,Example Data Quality Study,7343,Line-1-R1,This is Line-1,Targeted Proteomics,7352,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,0,counts,2506.0,24.0
2,7342,Example Data Quality Study,7343,Line-1-R1,This is Line-1,Targeted Proteomics,7352,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,0,counts,3154.0,24.0
3,7342,Example Data Quality Study,7344,Line-1-R2,This is Line-1,Targeted Proteomics,7353,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,0,counts,12513.0,24.0
4,7342,Example Data Quality Study,7344,Line-1-R2,This is Line-1,Targeted Proteomics,7353,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,0,counts,3244.0,24.0


Extract study name from EDD data and replace special characters/spaces with "_"

In [10]:
study_name = df["Study Name"][0]
study_name = re.sub('\W+','_', study_name )
study_name

'Example_Data_Quality_Study'

Create function for cleaning raw data

In [11]:
# Including 'Protocol' and 'Units'
excluded_columns = ['Study ID', 'Study Name', 'Study Description', 'Study Contact',
       'Line ID', 'Line Name', 'Line Description', 'Control', 'Strain(s)',
       'Carbon Source(s)', 'Line Experimenter', 'Line Contact', 'Media',
       'Protocol ID', 'Protocol Name', 'Assay ID', 
       'Measurement Updated', 'X Units', 'Y Units','Compartment']





In [12]:
def parse_edd_study(df):
    df = df.loc[:,~df.columns.isin(excluded_columns)]
    return df

In [13]:
df = parse_edd_study(df)

In [14]:
df.head()

,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours
0,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0
1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0
2,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0
3,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0
4,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0


In [15]:
# Dropdown options from unique protocol names
ALL = 'ALL'
def unique_values(array):
    unique = array.unique().tolist()
#     unique.sort()
#     unique.insert(0, ALL)
    return unique
unique_values(df["Protocol"])

['Targeted Proteomics']

In [16]:
# Create a class for colorful printing
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

## OPTIONAL: SPECIFY PROTOCOL

In [17]:
# Create dropdown options for different protocols. This will happen when there are multiple protocols in dataset.
if len(unique_values(df["Protocol"])) > 1:
    dropdown_protocol = widgets.Dropdown(options = unique_values(df["Protocol"]), description = "Protocol:" )
    # dropdown_type_abb
    print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}OPTIONAL:SPECIFY THE PROTOCOL FROM DROPDOWN MENU")
    print("If protocol is NOT specified, ONLY the FIRST protocol will be ANALYZED ")

    def final_df():
        user_change = dropdown_protocol.value
        updated_df = df[df["Protocol"] == user_change].reset_index(drop=True)
        return updated_df


    def df_update(protocol):
        updated_df = df[df["Protocol"] == protocol].reset_index(drop=True)
        display(updated_df.head())

    widgets.interact(df_update, protocol=dropdown_protocol)
    print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}AFTER PROTOCOL SELECTION, YOU MUST GO TO THE NEXT CELL")
    print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}Run --> Run Selected Cell and All Below")
else:
    df_protocol = df.copy()
    pass


In [18]:
# Run this cell and all below

In [19]:
# After sepecifying protocol, run this cell and all below
if len(unique_values(df["Protocol"])) > 1:
    df_protocol = final_df()

In [20]:
df_protocol.head()

,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours
0,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0
1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0
2,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0
3,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0
4,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0


In [21]:
# Checkpoint
# df_protocol.shape

In [22]:

# Extract only Replicate numbers from Assay Name
df0 = df_protocol['Assay Name'].str.extract(r'(?P<Assay_Name>[a-zA-Z0-9\_\- ]+)-(?P<Replicate_Num>[Rr]\d+)',expand=False)


In [23]:
df0.head()

,Assay_Name,Replicate_Num
0,Line-1,R1
1,Line-1,R1
2,Line-1,R1
3,Line-1,R2
4,Line-1,R2


***If there are no replicate numbers in column Replicete_Num, Assay Name will be kept same with raw data. 

In [24]:
if df0["Replicate_Num"].isnull().values.all(axis=0)== False:
    df1 = df0.merge(df_protocol, left_index=True,right_index=True)
elif df0["Replicate_Num"].isnull().values.all(axis=0)== True:
    df1=df_protocol.copy()
    df1 = df1.rename(columns={'Assay Name':'Assay_Name'})
else:
    pass

In [25]:
df1.head()

,Assay_Name,Replicate_Num,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours
0,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0
1,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0
2,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0
3,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0
4,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0


***If Formal Type is empty, do not try to extract anything. 

In [26]:
if df1["Formal Type"].isnull().values.all(axis=0)== True and df1["Measurement Type"].isnull().values.all(axis=0)== True:
    df1["Formal Type"] = "-"
    df1["Measurement Type"] = "-"
elif df1["Formal Type"].isnull().values.all(axis=0)== True and df1["Measurement Type"].isnull().values.all(axis=0)== False:
    df1["Formal Type"] = df1["Measurement Type"]
elif df1["Formal Type"].isnull().values.all(axis=0)== False and df1["Measurement Type"].isnull().values.all(axis=0)== True:
    df1["Measurement Type"] = df1["Formal Type"]
else:
    pass
    

In [27]:
df1.head()

,Assay_Name,Replicate_Num,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours
0,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0
1,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0
2,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0
3,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0
4,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0


In [28]:
try:
    df2 = df1['Formal Type'].str.extract(r'\|(?P<Type_ID>[a-zA-Z0-9]+)\|(?P<Type_Abb>[a-zA-Z0-9\_]+)?', expand=True)
    if df2["Type_ID"].isnull().values.all(axis=0)== True and df2["Type_Abb"].isnull().values.all(axis=0)== True:
        if df1['Formal Type'].isnull().values.all(axis=0)== False:
            df2["Type_ID"] = df1["Formal Type"]
            df2["Type_Abb"] = df1["Measurement Type"]
except AttributeError:
    df2=df1.copy()    

 

In [29]:
df2.head()

,Type_ID,Type_Abb
0,P00761,TRYP_PIG
1,P02769,ALBU_BOVIN
2,P00698,LYSC_CHICK
3,P00761,TRYP_PIG
4,P02769,ALBU_BOVIN


In [30]:
df3 = df1.merge(df2, left_index=True,right_index=True)


In [31]:
df3.head()

,Assay_Name,Replicate_Num,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours,Type_ID,Type_Abb
0,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0,P00761,TRYP_PIG
1,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0,P02769,ALBU_BOVIN
2,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0,P00698,LYSC_CHICK
3,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0,P00761,TRYP_PIG
4,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0,P02769,ALBU_BOVIN


In [32]:
df3.shape

(54, 11)

** If Type_Abb is NaN, replace NaN value with Measurement type value. 
** If Type_ID is NaN, replace NaN value with Formal type (This should work for Metabolomics data)

In [33]:
df3.loc[(pd.isnull(df3.Type_Abb)), "Type_Abb"] = df3["Measurement Type"]
df3.loc[(pd.isnull(df3.Type_ID)), "Type_ID"] = df3["Formal Type"]

In [34]:
df3.head()

,Assay_Name,Replicate_Num,Protocol,Assay Name,Formal Type,Measurement Type,Units,Value,Hours,Type_ID,Type_Abb
0,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,7209.0,24.0,P00761,TRYP_PIG
1,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,2506.0,24.0,P02769,ALBU_BOVIN
2,Line-1,R1,Targeted Proteomics,Line-1-R1,sp|P00698|LYSC_CHICK Lysozyme C,Lysozyme C,counts,3154.0,24.0,P00698,LYSC_CHICK
3,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P00761|TRYP_PIG Trypsin,Trypsin,counts,12513.0,24.0,P00761,TRYP_PIG
4,Line-1,R2,Targeted Proteomics,Line-1-R2,sp|P02769|ALBU_BOVIN Serum albumin,Serum albumin,counts,3244.0,24.0,P02769,ALBU_BOVIN


In [35]:
if "Replicate_Num" in df3.columns:
    df4 = df3[["Protocol","Assay_Name", "Replicate_Num", "Type_ID", "Type_Abb", "Measurement Type", "Hours","Units", "Value"]]
else:
    df4 = df3[["Protocol","Assay_Name", "Type_ID", "Type_Abb", "Measurement Type", "Hours","Units", "Value"]]


In [36]:
df4 = df4.applymap(lambda s:s.upper() if type(s) == str else s)

In [37]:
df4.head()

,Protocol,Assay_Name,Replicate_Num,Type_ID,Type_Abb,Measurement Type,Hours,Units,Value
0,TARGETED PROTEOMICS,LINE-1,R1,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,7209.0
1,TARGETED PROTEOMICS,LINE-1,R1,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,2506.0
2,TARGETED PROTEOMICS,LINE-1,R1,P00698,LYSC_CHICK,LYSOZYME C,24.0,COUNTS,3154.0
3,TARGETED PROTEOMICS,LINE-1,R2,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,12513.0
4,TARGETED PROTEOMICS,LINE-1,R2,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,3244.0


In [38]:
# df4["Type_ID"].isnull().values.all(axis=0)

In [39]:
df4.head()

,Protocol,Assay_Name,Replicate_Num,Type_ID,Type_Abb,Measurement Type,Hours,Units,Value
0,TARGETED PROTEOMICS,LINE-1,R1,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,7209.0
1,TARGETED PROTEOMICS,LINE-1,R1,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,2506.0
2,TARGETED PROTEOMICS,LINE-1,R1,P00698,LYSC_CHICK,LYSOZYME C,24.0,COUNTS,3154.0
3,TARGETED PROTEOMICS,LINE-1,R2,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,12513.0
4,TARGETED PROTEOMICS,LINE-1,R2,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,3244.0


In [40]:
# fill all NaN values in Units with "-"
df4["Units"] = df4["Units"].replace(np.nan, '-', regex=True)

In [41]:
df4.head()

,Protocol,Assay_Name,Replicate_Num,Type_ID,Type_Abb,Measurement Type,Hours,Units,Value
0,TARGETED PROTEOMICS,LINE-1,R1,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,7209.0
1,TARGETED PROTEOMICS,LINE-1,R1,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,2506.0
2,TARGETED PROTEOMICS,LINE-1,R1,P00698,LYSC_CHICK,LYSOZYME C,24.0,COUNTS,3154.0
3,TARGETED PROTEOMICS,LINE-1,R2,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,12513.0
4,TARGETED PROTEOMICS,LINE-1,R2,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,3244.0


In [42]:
# Selection options from unique assay names
none_exclude = 'None'
def unique_values_assay(array):
    unique = array.unique().tolist()
#     unique.sort()
    unique.insert(0, none_exclude)
    return unique
print(unique_values_assay(df4["Assay_Name"]))

['None', 'LINE-1', 'LINE-2', 'LINE-3']


## OPTIONAL: EXCLUDE ASSAY OR ASSAYS

In [43]:
# Create selection options to exclude assay/s....
if len(unique_values_assay(df4["Assay_Name"])) > 2:
    select_multi_assays = widgets.SelectMultiple(
                        options=unique_values_assay(df4["Assay_Name"]),
                        value=[none_exclude],
                        rows=10,
                        description="Exclude Assay(s): ",
                        style = {"description_width":"initial"},
                        disabled=False,
                        layout=Layout(width='30%', height='150px')
                    )   
    print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}OPTIONAL: EXCLUDE ASSAY(S) ")
    print("None means ALL the ASSAYS will be ANALYZED ")


    
    def final_df_assay():
        exc_assay_list = select_multi_assays.value
        if list(exc_assay_list) == ["None"]:
            updated_df = df4.copy()
            
        else:
            updated_df = df4[~df4["Assay_Name"].isin(exc_assay_list)].reset_index(drop=True)
        return updated_df    
        

    def df_update_assay(exc_assay_list):
        if list(exc_assay_list) == ["None"]:
            updated_df = df4.copy()
            display(updated_df.head())
        else:
            updated_df = df4[~df4["Assay_Name"].isin(exc_assay_list)].reset_index(drop=True)
            display(updated_df.head())

    widgets.interact(df_update_assay, exc_assay_list=select_multi_assays)
    
else:
    df_protocol_assay = df4.copy()
    pass
print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}AFTER ASSAY(S) SELECTION, YOU MUST GO TO THE NEXT CELL")
print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}Run --> Run Selected Cell and All Below")

OPTIONAL: EXCLUDE ASSAY(S) 
None means ALL the ASSAYS will be ANALYZED 


interactive(children=(SelectMultiple(description='Exclude Assay(s): ', index=(0,), layout=Layout(height='150px…

AFTER ASSAY(S) SELECTION, YOU MUST GO TO THE NEXT CELL
Run --> Run Selected Cell and All Below


In [44]:
# Run this cell and all below

In [45]:
if len(unique_values_assay(df4["Assay_Name"])) > 2:
    df_protocol_assay = final_df_assay()

In [46]:
# Test!!!!!

# df_protocol_assay["Units"][0] = "test"

In [47]:
df_protocol_assay.head()

,Protocol,Assay_Name,Replicate_Num,Type_ID,Type_Abb,Measurement Type,Hours,Units,Value
0,TARGETED PROTEOMICS,LINE-1,R1,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,7209.0
1,TARGETED PROTEOMICS,LINE-1,R1,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,2506.0
2,TARGETED PROTEOMICS,LINE-1,R1,P00698,LYSC_CHICK,LYSOZYME C,24.0,COUNTS,3154.0
3,TARGETED PROTEOMICS,LINE-1,R2,P00761,TRYP_PIG,TRYPSIN,24.0,COUNTS,12513.0
4,TARGETED PROTEOMICS,LINE-1,R2,P02769,ALBU_BOVIN,SERUM ALBUMIN,24.0,COUNTS,3244.0


In [48]:
# Check Points:

if df_protocol_assay["Units"].isnull().values.all(axis=0)== True:
    df_protocol_assay["Units"] = "-"
else:
    pass

if df_protocol_assay["Type_ID"].isnull().values.all(axis=0)== True:
    df_protocol_assay = df_protocol_assay.drop(columns="Type_ID")
    stat_df = df_protocol_assay.groupby(["Protocol","Assay_Name", "Type_Abb","Units","Hours"]).mean()
    stat_df.rename(columns={'Value': 'mean'}, inplace=True)
    stat_df["std"] = df_protocol_assay.groupby(["Protocol","Assay_Name", "Type_Abb","Units","Hours"]).std()
else:
    stat_df = df_protocol_assay.groupby(["Protocol","Assay_Name","Type_ID", "Type_Abb","Units","Hours"]).mean()
    stat_df.rename(columns={'Value': 'mean'}, inplace=True)
    stat_df["std"] = df_protocol_assay.groupby(["Protocol","Assay_Name", "Type_ID", "Type_Abb","Units","Hours"]).std()

stat_df["CV"] = stat_df['std'] / stat_df['mean']
stat_df = stat_df.round(2)

In [49]:
stat_df.head()

mean  \
Protocol            Assay_Name Type_ID Type_Abb   Units  Hours             
TARGETED PROTEOMICS LINE-1     P00698  LYSC_CHICK COUNTS 24.0    3257.00   
                                                         48.0    4885.50   
                                                         72.0    6351.15   
                               P00761  TRYP_PIG   COUNTS 24.0    9861.00   
                                                         48.0   14791.50   

                                                                    std    CV  
Protocol            Assay_Name Type_ID Type_Abb   Units  Hours                 
TARGETED PROTEOMICS LINE-1     P00698  LYSC_CHICK COUNTS 24.0    145.66  0.04  
                                                         48.0    218.50  0.04  
                                                         72.0    284.04  0.04  
                               P00761  TRYP_PIG   COUNTS 24.0   3750.49  0.38  
                                                         48.0   5625.74  0.38

In [50]:
# sum_df = stat_df.unstack(3)
sum_df = stat_df.copy()

In [51]:
sum_df.head()

mean  \
Protocol            Assay_Name Type_ID Type_Abb   Units  Hours             
TARGETED PROTEOMICS LINE-1     P00698  LYSC_CHICK COUNTS 24.0    3257.00   
                                                         48.0    4885.50   
                                                         72.0    6351.15   
                               P00761  TRYP_PIG   COUNTS 24.0    9861.00   
                                                         48.0   14791.50   

                                                                    std    CV  
Protocol            Assay_Name Type_ID Type_Abb   Units  Hours                 
TARGETED PROTEOMICS LINE-1     P00698  LYSC_CHICK COUNTS 24.0    145.66  0.04  
                                                         48.0    218.50  0.04  
                                                         72.0    284.04  0.04  
                               P00761  TRYP_PIG   COUNTS 24.0   3750.49  0.38  
                                                         48.0   5625.74  0.38

In [52]:
# len(unique_values(df["Protocol"]))
len(df_protocol["Protocol"].unique())

1

In [53]:
# Protocol name

user_identified_protocol= df_protocol["Protocol"].unique()[0].replace(" ","_")
print(user_identified_protocol)

# current date and time
now = datetime.now()
analysis_time = now.strftime("%Y%m%d-%H%M%S")
# print(analysis_time)



Targeted_Proteomics


In [54]:
# Renaming report files respectively such as subset_sum_data, subset_full_data
row_num_ori = df4.shape[0]
row_num_custom = df_protocol_assay.shape[0]


if row_num_ori == row_num_custom:
    # write summary report to a csv with a timestamps
    sum_df.to_csv("Sum_data_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".csv")
    # write full report to a csv with a timestamps
    df_protocol_assay.to_csv("Full_data_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".csv")
else:
     # write summary report to a csv with a timestamps
    sum_df.to_csv("Subset_Sum_data_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".csv")
    # write full report to a csv with a timestamps
    df_protocol_assay.to_csv("Subset_Full_data_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".csv")


# Visualization

In [55]:
import matplotlib.colors as mcolors
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly

In [56]:
tidy_data_v0 = stat_df.reset_index()
tidy_data_v0.rename(columns={'Hours': 'Time_Point'}, inplace=True)

tidy_data_v0.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38


In [57]:
tidy_data = tidy_data_v0.copy()

In [58]:
tidy_data["CV%"] = tidy_data["CV"]*100

In [59]:
tidy_data.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV,CV%
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04,4.0
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04,4.0
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04,4.0
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38,38.0
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38,38.0


In [60]:
# Visualization data, labels
l_type = list(tidy_data["Type_Abb"])
l_unit = list(tidy_data["Units"].str.lower())
tidy_data["type_units"] = [f'{x}*{y}' for x,y in list(zip(l_type, l_unit))]

# omics_types = tidy_data["Type_Abb"].unique()
omics_types_units = tidy_data["type_units"].unique()
type_units = tidy_data["type_units"].unique()
assay_types = tidy_data["Assay_Name"].unique()
time_points = [time for time in tidy_data.Time_Point.unique() if time in tidy_data.Time_Point.unique()]

In [61]:
# sort time points in case there are assays with different time points
time_points.sort()


In [62]:
# time_points

In [63]:
tidy_data.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV,CV%,type_units
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04,4.0,LYSC_CHICK*counts
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04,4.0,LYSC_CHICK*counts
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04,4.0,LYSC_CHICK*counts
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38,38.0,TRYP_PIG*counts
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38,38.0,TRYP_PIG*counts


In [64]:
len(tidy_data["type_units"].unique())

3

In [65]:
len(tidy_data["Type_Abb"].unique())

3

In [66]:
# y axis label with units
unit_list = [f'avg ('+unit.split("*")[1]+')' for unit in tidy_data["type_units"].unique() ]

In [67]:
# unit_list

In [68]:
# subplot titles
titles_subplots = [unit.split("*")[0] for unit in tidy_data["type_units"].unique() ]

In [69]:
# titles_subplots

# Bar Plot

In [70]:


subplot_rows= len(tidy_data["type_units"].unique())
subplot_cols = 1
tot_subplots = subplot_rows * subplot_cols

fig= make_subplots(
    rows=subplot_rows, cols=subplot_cols,
    shared_xaxes=False, shared_yaxes=False,
    horizontal_spacing=0.03, 
#     vertical_spacing=0.03,
    subplot_titles=titles_subplots,
#     x_title="Assay",
#     y_title="test_y_title",
#     changing row titles
#     row_titles=["avg"]*subplot_rows
)


for time_point in time_points:
    for i in range(len(tidy_data["type_units"].unique())):
        omics_type = omics_types_units[i]
        mean_vals = list()
        std_vals = list()
        for j in range(len(tidy_data["Assay_Name"].unique())):
            assay_type=assay_types[j]
            filt_val = tidy_data[
                (tidy_data["Time_Point"]==time_point)
                &
                (tidy_data["type_units"]==omics_type)
                &
                (tidy_data["Assay_Name"]==assay_type)
                    ]
            
            if len(filt_val)>0:
                mean_val = filt_val["mean"].values[0]
            else:
                mean_val = 0
            mean_vals.append(mean_val)
            
            if len(filt_val)>0:
                std_val = filt_val["std"].values[0]
            else:
                std_val = 0
            std_vals.append(std_val)
        
#         print(std_vals)
        fig.add_trace(
             go.Bar(
                x=assay_types,
                y=mean_vals,
                name=time_point,
                error_y=dict(
                type="data", 
                array=std_vals,
                visible=True,
                thickness=1,
                width=2
                ),
#                 hovertext = time_point
    #             mode="lines"
            ),
            row=(i) + 1, col=(1),
#             row=(i // subplot_cols) + 1, col=((i % subplot_cols) + 1),
            )
           
            
            
fig.update_layout(
    title_text=f'{user_identified_protocol.upper()} Data',
    barmode='group', height=240 * (subplot_rows), width=230 * 8,
    showlegend=False,
#     xaxis_title="x Axis Title",
#     yaxis_title="y Axis Title",
#     plot_bgcolor="white",
    font=dict(
        family="Arial, Tahoma, Helvetica",
        size=7,
        color="#404040",
    ),
)
fig.update_annotations(patch=dict(font=dict(size=10, color='#404040')))



for k in list(range(0, subplot_rows)):
    fig.update_yaxes(title_text=unit_list[k], row=k+1, col=1)



fig.update_yaxes( fixedrange=True)
fig.update_xaxes(patch=(dict(type= 'category')),fixedrange=True)

# fig.show(config={'displayModeBar': False})

pyo.plot(fig, filename = "Bar_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".html")




'Bar_Example_Data_Quality_Study_TARGETED_PROTEOMICS_20201124-153115.html'

# Line Plot 

In [71]:
# Line plot will be generated if there are are more than 1 time points

***Create a function for line plot: 

In [72]:
len(tidy_data.Time_Point.unique())

3

In [73]:
def line_plot(df_plot):
    subplot_rows= len(df_plot["type_units"].unique())
    subplot_cols = 1

    tot_subplots = subplot_rows * subplot_cols

    fig= make_subplots(
    rows=subplot_rows, cols=subplot_cols,
    shared_xaxes=False, shared_yaxes=False,
    horizontal_spacing=0.03, 
    #     vertical_spacing=0.03,
    subplot_titles=titles_subplots,
#         x_title="Time Point",
    #     y_title="test_y_title",
#     row_titles=["avg"]*subplot_rows
    )
    
    for j in range(len(df_plot["Assay_Name"].unique())):
        assay_type=assay_types[j]
        for i in range(len(df_plot["type_units"].unique())):
            omics_type = omics_types_units[i]
            mean_vals = list()
            std_vals=list()
            for time_point in time_points:
                filt_val = df_plot[
                    (df_plot["type_units"]==omics_type)
                    &
                    (df_plot["Assay_Name"]==assay_type)
                    &
                    (df_plot["Time_Point"]==time_point)
                    ]
                if len(filt_val)>0:
                    mean_val = filt_val["mean"].values[0]
                else:
                    mean_val = 0
                mean_vals.append(mean_val)
    
    
    
                if len(filt_val)>0:
                    std_val = filt_val["std"].values[0]
                else:
                    std_val = 0
                std_vals.append(std_val)
    
            fig.add_trace(
                go.Scatter(
                x=time_points,
                y=mean_vals,
                name=assay_type,
                error_y=dict(
                    type="data", 
                    array=std_vals,
                    visible=True,
                    thickness=1,
                    width=2    
                ),
                mode="lines"
            ),
            row=(i) + 1, col=(1),
#             row=(i // subplot_cols) + 1, col=((i % subplot_cols) + 1),
            )
           
    fig.update_layout(
        title_text=f'{user_identified_protocol.upper()} Data',
        barmode='group', height=240 * (subplot_rows), width=230 * 8,
        showlegend=False,
    #     plot_bgcolor="white",
        font=dict(
            family="Arial, Tahoma, Helvetica",
            size=7,
            color="#404040",
        ),
    )
    fig.update_annotations(patch=dict(font=dict(size=10, color='#404040')))

    for k in list(range(0, subplot_rows)):
        fig.update_yaxes(title_text=unit_list[k], row=k+1, col=1)
        
#     fig.update_yaxes( fixedrange=True)
    fig.update_xaxes(fixedrange=True)
#     fig.show(config={'displayModeBar': False})

       
    return pyo.plot(fig, filename = "Line_"+study_name+"_"+user_identified_protocol.upper()+"_"+analysis_time+".html")

In [74]:
if len(tidy_data.Time_Point.unique())>1:
    line_plot(tidy_data)
else:
    pass

**Create a function to generate scatter plots.**

**Condition: if there is no replicates, do not generate scatter plots. 

# Data Quality visuals:

In [75]:
# checkpoint: if assay names are only numerical, 
# Work around to change numerical assay names to categorical name, added letter "x"
# tidy_data["Assay_Name"] = [f"x{x}" for x in tidy_data["Assay_Name"]]

In [76]:
tidy_data.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV,CV%,type_units
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04,4.0,LYSC_CHICK*counts
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04,4.0,LYSC_CHICK*counts
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04,4.0,LYSC_CHICK*counts
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38,38.0,TRYP_PIG*counts
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38,38.0,TRYP_PIG*counts


In [77]:
def is_int(val):
    try:
        num = int(val)
    except ValueError:
        return False
    return True

if  is_int(tidy_data["Assay_Name"][0]):
    tidy_data["Assay_Name"] = [f"x{x}" for x in tidy_data["Assay_Name"]]
else:
    pass

In [78]:
tidy_data.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV,CV%,type_units
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04,4.0,LYSC_CHICK*counts
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04,4.0,LYSC_CHICK*counts
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04,4.0,LYSC_CHICK*counts
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38,38.0,TRYP_PIG*counts
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38,38.0,TRYP_PIG*counts


In [79]:
l_type = list(tidy_data["Type_Abb"])
l_unit = list(tidy_data["Units"])
h_text = [f'{x},{y}' for x,y in list(zip(l_type, l_unit))]

In [80]:
# h_text

## OPTIONAL: SPECIFY CUTOFF PEAK AREA 

In [81]:
# Create a slider to specify cutoff peak area

slider = widgets.IntSlider(min=0, max=max(tidy_data["mean"]), value=0, description='Cutoff Peak:',step=100, layout=widgets.Layout(width='75%', height='100px'))


print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}OPTIONAL: SPECIFY CUTOFF PEAK AREA , Default is ZERO for cutoff peak area  ")


def return_tidy_data0():
    user_cutoff = slider.value
    if user_cutoff == 0:
        tidy_data0 = tidy_data.copy()
    else:
        tidy_data0 = tidy_data[tidy_data["mean"]>=user_cutoff].reset_index(drop=True)
    return tidy_data0

def user_cutoff_value():
    return slider.value


 
def calculation_extracted_data():
    user_cutoff = slider.value
    if user_cutoff == 0:
        tidy_data0 = tidy_data.copy()
        new_data_counts = 0
    else:
        tidy_data0 = tidy_data[tidy_data["mean"]>=user_cutoff].reset_index(drop=True)
        new_data_counts = tidy_data0.shape[0]
    ori_data_counts = tidy_data.shape[0]
    perc_ori_data_counts = (new_data_counts*100)/ori_data_counts
    sum_cutoff_area = [new_data_counts,ori_data_counts,round(perc_ori_data_counts,2)]
    return sum_cutoff_area

def tidy_data_cutoff(user_cutoff):
    user_cutoff = slider.value
   
    sum_cutoff_area_handler = calculation_extracted_data()
    
#    Buttons
    b1 = widgets.Button(description="Subset Data", layout=widgets.Layout(width="30%"))
    b2 = widgets.Button(description="Full Data", layout=widgets.Layout(width="30%"))
    b3 = widgets.Button(description="Percentage %", layout=widgets.Layout(width="30%"))
    b4 = widgets.Button(description=str(sum_cutoff_area_handler[0]), layout=widgets.Layout(width="30%"))
    b5 = widgets.Button(description=str(sum_cutoff_area_handler[1]), layout=widgets.Layout(width="30%"))
    b6 = widgets.Button(description=str(sum_cutoff_area_handler[2])+"%", layout=widgets.Layout(width="30%"))    
    h1 = widgets.HBox(children=[b1,b2,b3])
    h2 = widgets.HBox(children=[b4,b5, b6])
    data_buttons  = widgets.VBox(children=[h1,h2])
    if user_cutoff == 0:
        tidy_data0 = tidy_data.copy()  
        display(data_buttons)
        display(tidy_data0.head())
    else:
        tidy_data0 = tidy_data[tidy_data["mean"]>=user_cutoff].reset_index(drop=True)
        display(data_buttons)
        display(tidy_data0.head())

widgets.interact(tidy_data_cutoff, user_cutoff = slider)

print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}AFTER CHANGING THE INTENSITY, YOU MUST GO TO THE NEXT CELL")
print(f"{bcolors.HEADER}{bcolors.BOLD}{bcolors.UNDERLINE}Run --> Run Selected Cell and All Below")

OPTIONAL: SPECIFY CUTOFF PEAK AREA , Default is ZERO for cutoff peak area  


interactive(children=(IntSlider(value=0, description='Cutoff Peak:', layout=Layout(height='100px', width='75%'…

AFTER CHANGING THE INTENSITY, YOU MUST GO TO THE NEXT CELL
Run --> Run Selected Cell and All Below


In [82]:
# Run this cell and all below

In [83]:
tidy_data_df = return_tidy_data0()

In [84]:
tidy_data_df.head()

,Protocol,Assay_Name,Type_ID,Type_Abb,Units,Time_Point,mean,std,CV,CV%,type_units
0,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,24.0,3257.00,145.66,0.04,4.0,LYSC_CHICK*counts
1,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,48.0,4885.50,218.50,0.04,4.0,LYSC_CHICK*counts
2,TARGETED PROTEOMICS,LINE-1,P00698,LYSC_CHICK,COUNTS,72.0,6351.15,284.04,0.04,4.0,LYSC_CHICK*counts
3,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,24.0,9861.00,3750.49,0.38,38.0,TRYP_PIG*counts
4,TARGETED PROTEOMICS,LINE-1,P00761,TRYP_PIG,COUNTS,48.0,14791.50,5625.74,0.38,38.0,TRYP_PIG*counts


In [85]:
# tidy_data_df.shape
# tidy_data.shape

In [86]:
cutoff_peak_area = user_cutoff_value()

In [87]:
cutoff_peak_area

0

In [88]:
# current date and time
now = datetime.now()
cutoff_peak_time = now.strftime("%Y%m%d-%H%M%S")

# Exporting dataset after cutoff peak area gets updated:

if user_cutoff_value() >0 :
    print(f"User changed cutoff peak area: {cutoff_peak_area} ,Please see the new csv file generated as trimmed_data_DATE.csv")
    tidy_data_df.to_csv("trimmed_data_"+cutoff_peak_time+".csv")


# Scatter, Violin plot visualization function

In [89]:
def data_quality_visual(df_dq):

# Creating labels for markers    
    l_type = list(tidy_data_df["Type_Abb"])
    l_unit = list(tidy_data_df["Units"].str.lower())
    h_text = [f'{x}, {y}' for x,y in list(zip(l_type, l_unit))]
    
    
#   Specifying colors in order to make assay colors same in scatter and violin 
    cols=list(mcolors.XKCD_COLORS.values())
    
        
    # Calculating overall Data Quality
    tidy_data_passed_20 = df_dq[df_dq["CV%"]<20]
    counts = df_dq.shape[0]
    counts_dq = tidy_data_passed_20.shape[0]
    data_quality = round((counts_dq/counts)*100,2)
    
    # Calculating Data Quality metric per assay

    assay_name_list = list(df_dq["Assay_Name"].unique())
    time_point_list = list(df_dq["Time_Point"].unique())
    assay_dq = []
    for assay in assay_name_list:
        for time in time_point_list:
            counts_at = df_dq[(df_dq["Assay_Name"]==assay)&(df_dq["Time_Point"]==time)].shape[0]
            assay_passed_20 = df_dq.loc[(df_dq["Assay_Name"]==assay)&(df_dq["Time_Point"]==time)&(df_dq["CV%"]<20)].shape[0]
            if counts_at == 0 : 
                pass
            elif assay_passed_20 == 0:
                assay_dq.append(0)
            else:
                assay_dq.append(round((assay_passed_20/counts_at)*100,2))    
    # Creating Data Quality metric dataframe (Assay Name, Time Point, DataPointCount, DataQuality% )

    df_dq_assay = df_dq.groupby(["Assay_Name", "Time_Point"]).size().reset_index(name='DataPointCounts')
    df_dq_assay["DataQuality%"] = assay_dq
    

    fig_dq = make_subplots(
        rows=3, cols=1,
        shared_yaxes=False,
        shared_xaxes=False,
        vertical_spacing=0.08,
#         y_title="Proteomics Data Quality",
        row_titles=["CV%"],
        specs=[[{"type": "scatter"}],
               [{"type": "table"}],
               [{"type": "table"}]]
        )

    # Add traces
    for assay in assay_name_list:
        fig_dq.add_trace(
                go.Scatter(
                        x=df_dq["mean"][df_dq["Assay_Name"]==assay], 
                        y=df_dq["CV%"][df_dq["Assay_Name"]==assay],
                        mode='markers',
                        hovertext=h_text,
                        marker=dict(color=cols[assay_name_list.index(assay)]),
                        name=assay,
                        ),
                row=1,col=1)

    fig_dq.add_trace(
        go.Table(
            columnwidth = [1,1],
            header=dict(
                    values=list(df_dq_assay.columns),
                    font=dict(size=15),
                    align="left"
                    ),
            cells=dict(
                    values=[df_dq_assay["Assay_Name"],
                            df_dq_assay["Time_Point"],
                            df_dq_assay["DataPointCounts"],
                            df_dq_assay["DataQuality%"]],
                    align = "left",
                    ) 
    
                    ),
            row=2, col=1
            )

    fig_dq.add_trace(
        go.Table(
            columnwidth = [80,100],
            header=dict(
                values=["Overall Data Quality%"],
                font=dict(size=20),
                align="left"
            ),
            cells=dict(
                values=[data_quality],
                align = "left") 

        ),
        row=3, col=1
    )
    
    
    fig_dq.add_shape(
        # Line Horizontal
            type="line",
            x0=0,
            y0=20,
            x1=df_dq["mean"].max(),
            y1=20,
            line=dict(
                color="Red",
                width=2,
                dash="dashdot",
            ),row=1,col=1
        )
    # make space for explanation / annotation
#     fig_dq.update_layout(
# #                         margin=dict(l=20, r=20, t=20, b=60),
#                          paper_bgcolor="LightSteelBlue")

    # add annotation
    fig_dq.add_annotation(dict(font=dict(family="Times New Roman",color='black',size=20),
                                        x=0,
                                        y=0.69,
                                        showarrow=False,
                                        text=f"DataQuality%: The percentage of the data is below 20% CV (coefficient variation) for each set of replicates. <br> The red line in the scatter plot corresponds to 20% CV, a commonly used cutoff for data quality. This may or may not be suitable for your application..",
                                        textangle=0,
                                        align="left",
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"
                           ))

    
    fig_dq.update_layout(
            height=2000,
            title_text=f'{user_identified_protocol.upper()} Data Quality'
#             title_text= f"DataQuality%: The percentage of the data is below 20% CV (coefficient variation) for each set of replicates. <br> The red line in the scatter plot corresponds to 20% CV, a commonly used cutoff for data quality. This may or may not be suitable for your application.."
            )

    
#     fig_dq.show(config={'displayModeBar': False})

#     py.plot(fig_dq, filename="scatter_violin_table.html")
    return pyo.plot(fig_dq, filename = "Scatter_"+study_name+"_"+user_identified_protocol.upper()+"_"+cutoff_peak_time+".html")

In [90]:
# Scatter Visualization step Data Quality per Time point: 

# If there are assay replicates, provide an option to modify cutoff peak area
if tidy_data["CV%"].isnull().all():
    pass
else:
    data_quality_visual(tidy_data_df)

In [91]:
# Violin Plot per Time point:

def violin_visual(df_dq):

    subplot_rows= len(tidy_data["Time_Point"].unique())
    subplot_cols = 1
    tot_subplots = subplot_rows * subplot_cols
    


# Creating labels for markers    
    l_type = list(tidy_data_df["Type_Abb"])
    l_unit = list(tidy_data_df["Units"].str.lower())
    h_text = [f'{x}, {y}' for x,y in list(zip(l_type, l_unit))]
    
    
#   Specifying colors in order to make assay colors same in scatter and violin 
    cols=list(mcolors.XKCD_COLORS.values())
    
        
    # Calculating overall Data Quality
    tidy_data_passed_20 = df_dq[df_dq["CV%"]<20]
    counts = df_dq.shape[0]
    counts_dq = tidy_data_passed_20.shape[0]
    data_quality = round((counts_dq/counts)*100,2)
    
    # Calculating Data Quality metric per assay
    assay_dq = []
    assay_name_list = list(df_dq["Assay_Name"].unique())
    time_point_list = list(df_dq["Time_Point"].unique())
#     Sort the time point list. 
    time_point_list.sort()
    
    
    
    for assay in assay_name_list:
        for time in time_point_list:
            counts_at = df_dq[(df_dq["Assay_Name"]==assay)&(df_dq["Time_Point"]==time)].shape[0]
            assay_passed_20 = df_dq.loc[(df_dq["Assay_Name"]==assay)&(df_dq["Time_Point"]==time)&(df_dq["CV%"]<20)].shape[0]
            if counts_at == 0 : 
                pass
            elif assay_passed_20 == 0:
                assay_dq.append(0)
            else:
                assay_dq.append(round((assay_passed_20/counts_at)*100,2))
                
    
    # Creating Data Quality metric dataframe (Assay Name, Time Point, DataPointCount, DataQuality% )

    df_dq_assay = df_dq.groupby(["Assay_Name", "Time_Point"]).size().reset_index(name='DataPointCounts')
    df_dq_assay["DataQuality%"] = assay_dq
    

#     Title Subplots with Time Point
    time_subplots = list(map(lambda x: "CV% Per Time Point(h): " + str(x) , time_point_list))

#     Make subplots

    fig_dq= make_subplots(
        rows=subplot_rows, cols=subplot_cols,
        shared_xaxes=False, shared_yaxes=False,
        horizontal_spacing=0.03, 
    vertical_spacing=0.03,
        subplot_titles=time_subplots,
#     x_title="Assay",
#     y_title="test_y_title",
#     changing row titles
#     row_titles=["avg"]*subplot_rows
)
    
    
    # Add traces
    for i in range(len(time_point_list)):
        time = time_point_list[i]
        for j in range(len(assay_name_list)):
            assay = assay_name_list[j]
            
    
    for i in range(len(time_point_list)):
        time = time_point_list[i]
        for j in range(len(assay_name_list)):
            assay = assay_name_list[j]
            try:
                max_CV = max(tidy_data_df["CV%"][(tidy_data_df["Assay_Name"]==assay)&(tidy_data_df["Time_Point"]==time)])
            except ValueError:
                max_CV = 0
            fig_dq.add_trace(
                    go.Violin(
                        x=df_dq["Assay_Name"][df_dq["Assay_Name"]==assay], 
                        y=df_dq["CV%"][(df_dq["Assay_Name"]==assay)&(df_dq["Time_Point"]==time)],
                        name=assay,
                        box_visible=True,
                        meanline_visible=True,
                        line_color="Black",
                        fillcolor=cols[assay_name_list.index(assay)],
#                         span = [0, 100], spanmode='manual'
#                         constrain the distribution shown on a violin plot to be within the bounds of the data that is being plotted
                        span = [0, max_CV], spanmode='manual'

                    ),

                row=i+1,col=1
                )

    
    
    
    fig_dq.update_layout(
            height=3000,
            title_text=f'{user_identified_protocol.upper()} Data Quality Per Time Point'
            )

#     fig_dq.show(config={'displayModeBar': False})

#     py.plot(fig_dq, filename="scatter_violin_table.html")
    return pyo.plot(fig_dq, filename = "Violin_"+study_name+"_"+user_identified_protocol.upper()+"_"+cutoff_peak_time+".html")

In [92]:
# Violin Visualization step Data Quality per Time point: 

# If there are assay replicates, provide an option to modify cutoff peak area
if tidy_data["CV%"].isnull().all():
    pass
else:
    violin_visual(tidy_data_df)